In [ ]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [ ]:

!pip install dash-bootstrap-components

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import sqlite3
from datetime import datetime

# Database Configuration
DB_FILE = "amazon_sales.db"
TABLE_NAME = "sales_data"

def load_data():
    """
    Load cleaned sales data from SQLite database into a Pandas DataFrame.
    """
    conn = sqlite3.connect(DB_FILE)
    query = f"SELECT * FROM {TABLE_NAME}"
    data = pd.read_sql_query(query, conn)
    conn.close()

    # Convert 'date' column to datetime type if it exists
    if 'date' in data.columns:
        data['date'] = pd.to_datetime(data['date'])

    return data

# Initialize Dash App
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LUX])

# Load initial data
data = load_data()

# App Layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("📊 Amazon Sales Dashboard", className="text-center mb-4"), width=12)
    ]),

    dbc.Row([
        dbc.Col([
            html.H4("Filters", className="mb-3"),
            dcc.Dropdown(
                id='column-selector',
                options=[
                    {'label': 'Total Revenue', 'value': 'total revenue'},
                    {'label': 'Quantity', 'value': 'qty'},
                ],
                value='total revenue',
                clearable=False,
                className="mb-3"
            ),
            dcc.DatePickerRange(
                id='date-range',
                min_date_allowed=data['date'].min() if 'date' in data.columns else None,
                max_date_allowed=data['date'].max() if 'date' in data.columns else None,
                start_date=data['date'].min() if 'date' in data.columns else None,
                end_date=data['date'].max() if 'date' in data.columns else None,
                className="mb-3"
            )
        ], width=3),

        dbc.Col([
            dbc.Row([
                dbc.Col(dbc.Card([
                    dbc.CardBody([
                        html.H5("Total Revenue", className="card-title"),
                        html.H3(id="total-revenue", className="card-text")
                    ])
                ]), width=4),
                dbc.Col(dbc.Card([
                    dbc.CardBody([
                        html.H5("Total Orders", className="card-title"),
                        html.H3(id="total-orders", className="card-text")
                    ])
                ]), width=4),
                dbc.Col(dbc.Card([
                    dbc.CardBody([
                        html.H5("Average Order Value", className="card-title"),
                        html.H3(id="avg-order", className="card-text")
                    ])
                ]), width=4),
            ])
        ], width=9)
    ], className="mb-4"),

    dbc.Row([dbc.Col(dcc.Graph(id='sales-trend'), width=12)], className="mb-4"),
    dbc.Row([dbc.Col(dcc.Graph(id='sales-bar-chart'), width=12)], className="mb-4"),
    dbc.Row([
        dbc.Col(dcc.Graph(id='sales-pie-chart'), width=6),
        dbc.Col(dcc.Graph(id='daily-distribution'), width=6),
    ], className="mb-4"),

], fluid=True)

# **🔹 Callbacks for Summary Cards**
@app.callback(
    [Output('total-revenue', 'children'),
     Output('total-orders', 'children'),
     Output('avg-order', 'children')],
    [Input('column-selector', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date')]
)
def update_summary_cards(selected_column, start_date, end_date):
    filtered_data = data.copy()
    if start_date and end_date:
        filtered_data = filtered_data[
            (filtered_data['date'] >= start_date) &
            (filtered_data['date'] <= end_date)
        ]

    total_revenue = f"${filtered_data['total revenue'].sum():,.2f}"
    total_orders = len(filtered_data)
    avg_order = f"${filtered_data['total revenue'].mean():,.2f}"
    return total_revenue, total_orders, avg_order

# **🔹 Sales Trend Line Chart**
@app.callback(
    Output('sales-trend', 'figure'),
    [Input('column-selector', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date')]
)
def update_sales_trend(selected_column, start_date, end_date):
    filtered_data = data.copy()
    if start_date and end_date:
        filtered_data = filtered_data[
            (filtered_data['date'] >= start_date) &
            (filtered_data['date'] <= end_date)
        ]

    daily_data = filtered_data.groupby('date')[selected_column].sum().reset_index()
    return px.line(daily_data, x='date', y=selected_column, title=" Sales Trend")

# **🔹 Sales Bar Chart (Order ID vs Revenue)**
@app.callback(
    Output('sales-bar-chart', 'figure'),
    [Input('column-selector', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date')]
)
def update_bar_chart(selected_column, start_date, end_date):
    filtered_data = data.copy()
    if start_date and end_date:
        filtered_data = filtered_data[
            (filtered_data['date'] >= start_date) &
            (filtered_data['date'] <= end_date)
        ]

    return px.bar(filtered_data, x='order id', y=selected_column,
                  title=" Sales Data Per Order",
                  labels={selected_column: "Sales Amount"})

# **🔹 Pie Chart (Category-wise Sales)**
@app.callback(
    Output('sales-pie-chart', 'figure'),
    [Input('column-selector', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date')]
)
def update_pie_chart(selected_column, start_date, end_date):
    filtered_data = data.copy()
    if start_date and end_date:
        filtered_data = filtered_data[
            (filtered_data['date'] >= start_date) &
            (filtered_data['date'] <= end_date)
        ]

    category_data = filtered_data.groupby('category')[selected_column].sum().reset_index()
    return px.pie(category_data, names='category', values=selected_column,
                  title=" Sales Distribution by Category")

# **🔹 Daily Sales Distribution Box Plot**
@app.callback(
    Output('daily-distribution', 'figure'),
    [Input('column-selector', 'value'),
     Input('date-range', 'start_date'),
     Input('date-range', 'end_date')]
)
def update_daily_distribution(selected_column, start_date, end_date):
    filtered_data = data.copy()
    if start_date and end_date:
        filtered_data = filtered_data[
            (filtered_data['date'] >= start_date) &
            (filtered_data['date'] <= end_date)
        ]

    return px.box(filtered_data, x='date', y=selected_column,
                  title=" Daily Sales Distribution")

# **Run the Dash App**
if __name__ == '__main__':
    app.run_server(debug=True)


<ipython-input-6-a84f1ea40f24>:25: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



<IPython.core.display.Javascript object>